# Imports

In [ ]:
from keras import callbacks
from keras.models import Sequential
from keras.layers import Activation,Flatten,Dense,Dropout,Embedding,Bidirectional,LSTM
from keras.optimizers import Adam,SGD
import matplotlib.pyplot as plt

import pickle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Bidirectional,SpatialDropout1D
from keras.layers.embeddings import Embedding

from sklearn.model_selection import KFold
import gc
import numpy as np
import keras.backend as K

import numpy as np
import pandas as pd
import seaborn as sb
import math

import os
import re
import glob
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout
#from kaggle_datasets import KaggleDatasets
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading and preparation

In [ ]:
train_path = r'/content/drive/MyDrive/FakeNews/FakeNewsDetection Dataset/Whole Dataset'
# test_path = r'/content/drive/MyDrive/FakeNews/FakeNewsDetection Dataset/Stratified K Fold/Test Set'
all_train_files = glob.glob(train_path + "/*.csv")
# all_test_files = glob.glob(test_path + "/*.csv")

In [ ]:
FoldIndex = 0
rootDir = "/content/drive/My Drive/FakeNews/Without SMOTE Results/BERT/Final Run/" #change here for each run, i.e. Run1, Run2
# modelFileName = rootDir+"model_fold"+str(FoldIndex)+'.pkl'
modelFileName = rootDir+"final_model_0"+'.pkl'
# prevModelFileName = rootDir+"model_fold"+str(FoldIndex-1)+'.pkl'
model_type = "bert"
outputDir = rootDir+model_type

In [ ]:
train_texts=[]
train_labels= []
# test_texts=[]
# test_labels = []
for i in range(0, len(all_train_files)):
  train_file = pd.read_csv(all_train_files[i],encoding='utf-8')
  # test_file = pd.read_csv(all_test_files[i],encoding='utf-8')
  train_texts.append(train_file['text'].tolist())
  train_labels.append(train_file['labels'].tolist())
  # test_texts.append(test_file['Description'].tolist())
  # test_labels.append(test_file['Label'].tolist())

In [ ]:
print(len(train_texts)," ",len(train_labels))
print(len(train_texts[FoldIndex])," ",len(train_labels[FoldIndex]))

1   1
3187   3187


In [ ]:
"""Sanity check!!"""
print("fold: ",FoldIndex)
print("Number of train datapoints in this fold:", len(train_texts[FoldIndex]))
print(train_texts[FoldIndex][3][:512])
# print("Number of test datapoints in this fold:", len(test_texts[FoldIndex]))
# print(test_texts[FoldIndex][3][:512])

fold:  0
Number of train datapoints in this fold: 3187
 সাধারণ পাশ সরক রেশন কার্ড লকডাউন  টাকা দেব সরক করোন জের লকডাউন মানুষ বাঁচা এগিয় সরকার৷ রেশন কার্ড বরাদ্দ  টাকা৷ রেশন কার্ড পরিব পিছু  টাকা দেব বিপর্যয় মোকাবিলা বাহিনী৷ রেশন কার্ড পরিব পিছু  টাকা দেব বিপর্যয় মোকাবিলা বাহিনী৷ পুরসভা পঞ্চায়ত অঞ্চল সকল টাকা পৌঁ যাবে৷ দেশব্যাপী লকডাউন পরপর ঘোষণা বিহ সরকার৷ পুরসভা পঞ্চায়ত অঞ্চল সকল টাকা পৌঁ যাবে৷ দেশব্যাপী লকডাউন পরপর ঘোষণা বিহ সরকার৷


In [ ]:
math.ceil(len(train_texts[FoldIndex])/8)

399

In [ ]:
def makeDatasetDFs(textList, labelList):
  ll=[]
  for i in range(len(textList)):
    ll.append([textList[i],labelList[i]])
  return pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
df_train = makeDatasetDFs(train_texts[FoldIndex], train_labels[FoldIndex])
# df_test = makeDatasetDFs(test_texts[FoldIndex], test_labels[FoldIndex])

In [ ]:
df_train.head()

,text,labels
0,দেশ হৃদরোগজনিত মৃত্যু অসুস্থত চিত্র অত্যন্ত উ...,1
1,তাওয় উঠ আ জানি বল ডিগ্রি সেলসিয়াস টেম্পারেচ ...,0
2,সিলেট একদিন অর্ধশত করোনা রোগী শনাক্ত সিলেট বি...,1
3,সাধারণ পাশ সরক রেশন কার্ড লকডাউন টাকা দেব সর...,1
4,করোনা মার্কিন অভিনেত্রীর মৃত্যু করোনা মার্কিন...,1


In [ ]:
# df_test.head()

#Model Preparation

In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 12.6 MB/s 
     |████████████████████████████████| 1.2 MB 23.6 MB/s 
     |████████████████████████████████| 1.8 MB 43.5 MB/s 
     |████████████████████████████████| 8.0 MB 26.9 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
     |████████████████████████████████| 124 kB 53.8 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 2.6 MB 42.3 MB/s 
     |████████████████████████████████| 264 kB 43.1 MB/s 
     |████████████████████████████████| 895 kB 38.3 MB/s 
     |████████████████████████████████| 636 kB 39.9 MB/s 
     |████████████████████████████████| 133 kB 29.2 MB/s 
     |████████████████████████████████| 170 kB 57.9 MB/s 
     |████████████████████████████████| 138 kB 52.7 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████

# BERT

In [ ]:
model_name = "bert-base-multilingual-cased"

train_args = {
    "max_seq_length": 512,
    "num_train_epochs": 15,
    "fp16" : True,
    # "scheduler" : "polynomial_decay_schedule_with_warmup",
    # "polynomial_decay_schedule_lr_end": 1e-7,
    # "polynomial_decay_schedule_power": 2.0,
    "scheduler" : "cosine_schedule_with_warmup",
    "cosine_schedule_num_cycles": 0.5,
    #"scheduler" : "constant_schedule",
    "learning_rate": 1e-5,
    "train_batch_size": 8,
    "weight_decay": 1,
    "reprocess_input_data": False,
    "overwrite_output_dir": True,
    "use_cached_eval_features": False,
    "no_save": False,
    "output_dir": outputDir,
    "best_model_dir": modelFileName,
    "use_early_stopping": False,
    "logging_steps": math.ceil(len(train_texts[FoldIndex])/8),
    # "early_stopping_delta": 0.0,
    # "early_stopping_metric": "eval_loss",
    # "early_stopping_metric_minimize" : True,
    # "early_stopping_patience" : 2,
    "evaluate_during_training": False,
    # "evaluate_during_training_steps": 4,
    "wandb_project": "BERT_final", #change here for each separate run
    "wandb_kwargs": {"name": "final_"+str(FoldIndex),"entity" :'fakenewscovid'}, #change here for each separate run
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    # "eval_batch_size": 8,
    "evaluate_during_training_verbose" : True
}

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5887814108302047395, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11344216064
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8071270781652927882
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs

cuda_available = torch.cuda.is_available()
model = ClassificationModel(model_type, model_name, args=train_args, use_cuda=cuda_available)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
model.get_named_parameters()


['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerNorm.weight',
 'bert.encoder.layer

In [ ]:
history = model.train_model(df_train)

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarnin

In [ ]:
print(history)

(5985, 0.17032788220467493)


In [ ]:
pickle.dump(model, open(modelFileName, 'wb'))

In [ ]:
# if False:
#   result, model_outputs, wrong_predictions = model.eval_model(df_test)
#   predictions = np.argmax(model_outputs,axis=1)

In [ ]:
# if False:
#   print("model_output:\n\n",model_outputs)
#   print("\n\npredictions: \n\n",predictions)

In [ ]:
# if False:
#   from sklearn.metrics import accuracy_score
#   print(predictions)
#   print(accuracy_score(df_test['labels'],predictions))

In [ ]:
# predictions, raw_outputs = model.predict(df_test['text'].tolist())
predictions, raw_outputs = model.predict(df_train['text'].tolist())

In [ ]:
if False:
  print(predictions)

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, precision_score
# acc = accuracy_score(df_test['labels'],predictions)
acc = accuracy_score(df_train['labels'],predictions)
# recall = recall_score(df_test['labels'],predictions)
recall = recall_score(df_train['labels'],predictions)
# f1 = f1_score(df_test['labels'],predictions)
f1 = f1_score(df_train['labels'],predictions)
# precision = precision_score(df_test['labels'],predictions)
precision = precision_score(df_train['labels'],predictions)
# tn, fp, fn, tp = confusion_matrix(df_test['labels'],predictions).reshape(-1)
tn, fp, fn, tp = confusion_matrix(df_train['labels'],predictions).reshape(-1)

In [ ]:
print("acc: ",acc)
print("recall: ",recall)
print("f1: ",f1)
print("precision: ",precision)
print("tp: ",tp,"\nfn: ", fn,"\nfp: ", fp,"\ntn: ",tn)

acc:  0.9965484781926577
recall:  0.9996133023975251
f1:  0.9978768577494692
precision:  0.9961464354527938
tp:  2585 
fn:  1 
fp:  10 
tn:  591


In [ ]:
if FoldIndex == 0:
  resultFile = pd.DataFrame([],columns=['Fold No', 'Accuracy','Recall','Precision','F1_Score','TP','TN','FP','FN'])
else:
  resultFile = pd.read_csv(rootDir+'Results.csv',encoding='utf-8')
# resultFile = pd.read_csv(rootDir+'Results.csv',encoding='utf-8')
print(resultFile)

data = {'Fold No': str(FoldIndex), 'Accuracy':acc, 'Recall':recall, 'Precision':precision, 'F1_Score':f1,
          'TP':tp, 'TN':tn, 'FP':fp, 'FN':fn }

resultFile=resultFile.append(data,ignore_index=True,sort=False)
print(resultFile)


resultFile.to_csv(rootDir+'Results.csv', index=False, encoding='utf-8')

Empty DataFrame
Columns: [Fold No, Accuracy, Recall, Precision, F1_Score, TP, TN, FP, FN]
Index: []
  Fold No  Accuracy    Recall  Precision  F1_Score    TP   TN  FP FN
0       0  0.996548  0.999613   0.996146  0.997877  2585  591  10  1


In [ ]:
del model

In [ ]:
import json
with open(rootDir+'hyperparameters.txt', 'w') as file:
     file.write(json.dumps(train_args)) 